In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索ツールの定義
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
tools = [tool]

In [3]:
tool.invoke("Langgraphのノードとは？")

[{'url': 'https://zenn.dev/pharmax/articles/8796b892eed183',
  'content': 'Nodeは、Graph上で実際のステップを実行するコンポーネントです。例えば実際にLangChainを使用してLLMのモデルを呼び出す処理は、Node上で行います。\n\nNodeでは、`(State、Config) -> 変更するState`というシグネチャを持つ関数を使うことができます。 [...] | コンポーネント | 説明 |\n| --- | --- |\n| Graph | LangGraphの中核となる構成要素で、各NodeとEdgeの集合体です。 |\n| State | ノード間の遷移の際に保持される情報で、各ノードが参照および更新します。 |\n| Node | グラフ内の個々のステップや状態を表す要素で、特定のアクションやチェックポイントとして機能します。 |\n| Edge | ノード間の接続を表し、遷移の条件やアクションを定義します。条件付きエッジなど、特定のロジックに基づいて遷移を制御できます。 |\n\n## Graph\n\nGraphは、LangGraphの中核となるグラフ全体を管理するためのコンポーネントです。基本的な使い方としては、`StateGraph`というクラスを使い、後述するStateとセットで初期化します。\n\n`StateGraph`\n\n（StateGraphの宣言例）'},
 {'url': 'https://qiita.com/KUROMAGOORO/items/b37c28dbddee4314153e',
  'content': '| LangGraph の要素 | 説明 |\n| --- | --- |\n| State | 会話履歴や処理結果などを保持し、ノード間で共有・更新出来る。これにより、長い対話や複数ステップの処理が安定して行える。 |\n| Node | 実際の処理を行う関数やエージェント。Stateを参照・更新出来る |\n| Edge | ノード間の接続。「あらかじめ定義したノードに遷移」「条件に応じて次のノードを選択」「一定条件で繰り返す」など柔軟なノード遷移の設定が可能。Edge を定義せず LangGraph の goto の関数

In [4]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

# Stateクラスの定義
class State(TypedDict):
    messages: Annotated[list, add_messages]

# グラフのインスタンスを作成
graph_builder = StateGraph(State)

# 言語モデルの定義
llm = ChatOpenAI(model_name=MODEL_NAME)

# 変更点：ツール定義の紐づけ
llm_with_tools = llm.bind_tools(tools)

# チャットボットノードの作成
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# グラフにチャットボットノードを追加
graph_builder.add_node("chatbot", chatbot)

In [5]:
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ツールノードの作成
tool_node = ToolNode(tools)

# グラフにツールノードを追加
graph_builder.add_node("tools", tool_node)

# 条件付エッジの作成
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition, # ツール呼出と判断したらツールノードを呼ぶ
)

# ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
# ツールからチャットボットへの戻りエッジを作成
graph_builder.add_edge("tools", "chatbot")

# 開始ノードの指定
graph_builder.set_entry_point("chatbot")

# 記憶を持つ実行可能なステートグラフの作成
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [6]:
# グラフの実行と結果の表示
def stream_graph_updates(user_input: str):
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# チャットボットのループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(user_input)

こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？
1足す2は？
1足す2は3です。何か他に知りたいことはありますか？
1メートル以上の魚は？

[{"url": "https://www.zukan-bouz.com/syu/%E3%82%AA%E3%82%AA%E3%83%8B%E3%83%99", "content": "## 基本情報\n\n  宮城県でも見つかっているが、千葉県から九州までの暖かい海域で水揚げがある。古くは九州などが主な産地であったが、近年相模湾や駿河湾でもかなりの水揚げをみる。豊洲などの関東の市場でも標準和名のオオニベが定着している。  \n  天然のほか、熊本県、宮崎県で「みなみすずき」という商品名で養殖していた。  \n  １m以上になる大型魚でイヤミのない魚なので知名度はないものの、徐々に人気を得ている。  \n  関東の市場でも普通になっていて、切り身などで小売店に並ぶこともある。  \n  鮮度がいいと刺身にもなるが、安くておいしいのでフライや煮つけなどの総菜魚として利用価値が高い。  \n  珍魚度 普通の食用魚である。ただし大型魚なので切身で売られていることが多い。丸のままの状態で手に入れるためには努力を要す。\n\n  ## 水産基本情報 [...] Jewfish   \n  言語英語　場所オーストラリア\n\n  日本銀身   \n  言語中国語　場所臺灣　備考台湾魚類資料庫\n\n  ## 学名\n\n  Argyrosomus japonicus (Temminck and Schlegel, 1844)\n\n  ## 漢字・学名由来\n\n  漢字 大鮸　Standard Japanese name / Oonibe  \n  由来・語源 ニベ科で非常に大型になる。  \n  〈ニベ科ニベ屬　オホニベ　……著しく大型となる。〉『日本産魚類検索』（岡田彌一郎、松原喜代松　三省堂　初版1938）  \n  シーボルト、日本動物誌／ファウナ・ヤポニカ（Fauna Japonica ）  フィリップ・フランツ・フォン・シーボルトとその後継者、ハインリヒ・ビュルゲルなどが標本を持ち帰り、川原慶賀（江戸時代の長崎の絵師）が図を書いたもののひとつ。オランダライデン王立自然史博物館のシュレーゲルとテミンクが記載。